# 目的
音声データのスペクトログラムを入力として、性別および年齢を予測するCNNの学習.  
性別予測器と年齢予測器をそれぞれ学習する.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import cv2

from keras.applications.vgg16 import VGG16
from keras.models import Sequential, Model
import keras.backend as K
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.optimizers import SGD, Adam

from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.utils import plot_model
from keras.callbacks import TensorBoard
from keras import callbacks

from keras.utils import np_utils
from keras.layers.core import Lambda

from keras.utils.vis_utils import model_to_dot
from keras import callbacks

import tensorflow as tf

from keras.models import load_model
import pickle

from sklearn import preprocessing
from keras.utils import np_utils
from tqdm import tnrange, tqdm_notebook, tqdm

from keras.utils import multi_gpu_model
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


## Load data

In [ ]:
x_train = np.expand_dims(np.load('corevo/features/spectrogram/x_train.npy'), 3)
y_train = np.load('corevo/features/spectrogram/y_train.npy')

    
N, w, h, _ = x_train.shape
y_train_hot = np_utils.to_categorical(y_train)

label_dict = {'MA_CH':0, 'MA_AD':1, 'MA_EL':2,'FE_CH':3, 'FE_AD':4, 'FE_EL':5}

## Build model

In [3]:
def get_sex_estimator():
    model = Sequential()
    model.add(Conv2D(32, (4, 4), strides=(1,1),  input_shape=(w, h, 1), activation='relu', name='first_conv_layer'))
    model.add(Conv2D(32, (4, 4), strides=(1, 1), activation='relu'))
    model.add(MaxPool2D())
    
    model.add(Conv2D(32, (4, 4), strides=(1, 1), activation='relu'))
    model.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', name='last_conv_layer'))
    model.add(MaxPool2D())
    
    model.add(Flatten())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

def get_age_estimator():
    model = Sequential()
    model.add(Conv2D(32, (4, 4), strides=(1,1),  input_shape=(w, h, 1), activation='relu', name='first_conv_layer'))
    model.add(Conv2D(32, (4, 4), strides=(1, 1), activation='relu'))
    model.add(MaxPool2D())
    
    model.add(Conv2D(32, (4, 4), strides=(1, 1), activation='relu'))
    model.add(Conv2D(16, (3, 3), strides=(1, 1), activation='relu', name='last_conv_layer'))
    model.add(MaxPool2D())
    
    model.add(Flatten())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

sex_estimator = get_sex_estimator()
age_estimator = get_age_estimator()

In [4]:
# Relable data
y_sex = np.where(y_train<3, 0, 1)
y_age = y_train % 3

## 性別予測器の学習

In [6]:
es_cb = callbacks.EarlyStopping(monitor='val_loss', patience=1, verbose=0, mode='auto')
sex_estimator.fit(x_train,np_utils.to_categorical(y_sex,2), batch_size=128,epochs=10, validation_split=0.05, callbacks=[es_cb])

Train on 158838 samples, validate on 8360 samples
Epoch 1/10
158838/158838 [==============================] - 46s 289us/step - loss: 0.6409 - acc: 0.6036 - val_loss: 0.5451 - val_acc: 0.7079
Epoch 2/10
158838/158838 [==============================] - 40s 249us/step - loss: 0.4843 - acc: 0.7521 - val_loss: 0.4405 - val_acc: 0.7712
Epoch 3/10
158838/158838 [==============================] - 40s 249us/step - loss: 0.4107 - acc: 0.7968 - val_loss: 0.4234 - val_acc: 0.7854
Epoch 4/10
158838/158838 [==============================] - 40s 249us/step - loss: 0.3626 - acc: 0.8244 - val_loss: 0.4015 - val_acc: 0.8050
Epoch 5/10
158838/158838 [==============================] - 39s 248us/step - loss: 0.3272 - acc: 0.8444 - val_loss: 0.4038 - val_acc: 0.8141


In [7]:
# save model
sex_estimator.save('models/sex_estimator.h5')

## 年齢予測器の学習

In [9]:
es_cb = callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto')
age_estimator.fit(x_train,np_utils.to_categorical(y_age,3), batch_size=128,epochs=20, validation_split=0.05, callbacks=[es_cb])
# save model
age_estimator.save('models/age_estimator.h5')

Train on 158838 samples, validate on 8360 samples
Epoch 1/20
158838/158838 [==============================] - 40s 249us/step - loss: 0.2927 - acc: 0.8800 - val_loss: 0.5279 - val_acc: 0.8077
Epoch 2/20
158838/158838 [==============================] - 40s 249us/step - loss: 0.2758 - acc: 0.8890 - val_loss: 0.5550 - val_acc: 0.8132
Epoch 3/20
158838/158838 [==============================] - 40s 250us/step - loss: 0.2591 - acc: 0.8963 - val_loss: 0.5458 - val_acc: 0.8150


## Validate model

In [15]:
sex_estimator = load_model('models/sex_estimator.h5')
age_estimator = load_model('models/age_estimator.h5')

with open('corevo/features/spectrogram/validation.pkl', mode='rb') as f:
    validation = pickle.load(f)

n = len(validation)
ans = []
score = 0
confusion = np.zeros((6,6))

for cases, label_true in tqdm(validation):
    predicts = np.zeros(6)
    
    for case in cases:
        x = case.reshape(1, 128, 94, 1)
        
        pred_sex = sex_estimator.predict(x)
        pred_age = age_estimator.predict(x)
        
        pred_arg_sex = np.argmax(pred_sex)
        pred_arg_age = np.argmax(pred_age)
        
        label_pred = pred_arg_sex * 3 + pred_arg_age

        predicts[label_pred] += 1
        
    label_pred_final = np.argmax(predicts)
    
    # evaluate
    if label_pred_final == label_true:
        score += 1
        
    confusion[label_true, label_pred_final] += 1
    
#  compute validation accuracy
print(f'accuracy: {score}/{n} - {score/n*100}%' )

100%|██████████| 1891/1891 [00:26<00:00, 71.32it/s]

accuracy: 1513/1891 - 80.01057641459545%


# Predict on test data
## Load test data

In [9]:
with open('corevo/features/spectrogram/test.pkl', mode='rb') as f:
    f_names, test_data = pickle.load(f)

sex_estimator = load_model('models/sex_estimator.h5')
age_estimator = load_model('models/age_estimator.h5')

n = len(test_data)

# label_sex = ['MA', 'FE']
# label_age = ['CH', 'AD', 'EL']
label_dict = {'MA_CH':0, 'MA_AD':1, 'MA_EL':2,'FE_CH':3, 'FE_AD':4, 'FE_EL':5}

## Creat prediction result for submission

In [12]:
ans = []

for cases in tqdm(test_data):
    predicts = np.zeros(6)

    for case in cases:
        x = case.reshape(1, 128, 94, 1)
        
        pred_sex = sex_estimator.predict(x)
        pred_age = age_estimator.predict(x)
        
        pred_arg_sex = np.argmax(pred_sex)
        pred_arg_age = np.argmax(pred_age)
        
        
        label_pred = pred_arg_sex * 3 + pred_arg_age

        predicts[label_pred] += 1
        
    label_pred_final = np.argmax(predicts)
    pred_final_str = list(label_dict.keys())[label_pred_final]
    
    ans.append(pred_final_str)  
    
ans = np.array(ans)
f_names = np.array([f[:-4] for f in f_names])

out = np.vstack((f_names, ans)).T

np.savetxt('submit_0121_1616.tsv', out, fmt='%s', delimiter='\t')

100%|██████████| 17888/17888 [03:22<00:00, 88.43it/s]
